In [30]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import statsmodels.api as sm
#import matplotlib

In [2]:
url = 'https://dogvacay.com/dog-boarding--az--phoenix'

In [3]:
r = requests.get(url)
time.sleep(1)

In [4]:
r.status_code

200

In [5]:
soup = BeautifulSoup(r.text)

In [6]:
sitters = soup.findAll(lambda tag:tag.name == "div" and \
                        len(tag.attrs) == 1 and \
                        tag.attrs.keys()[0] == "search-list-item")

In [7]:
len(sitters)

15

In [19]:
sitter = sitters[0]
times = []
fees = []
reviews = []
repeats = []
city = []
name = []
rating = []
service = []

for sitter in sitters:
    rtwrap = sitter.findAll('li', {'class': 'dv-search-badge-item'})
    if len(rtwrap) > 0:
        response_time = rtwrap[0].text.strip('\n')[18:]
    else:
        response_time = -1
    fee = sitter.findAll('span', {'class': "dv-host-list-item__price__amount"})[0].text.strip()[1:]
    #if len(price) > 0:
    #    fee = price[0].text.strip()[1:]
    #else:
    #    fee = 0
    review = sitter.find('span', {'class': 'dv-host-review'})
    if review is not None and len(review) > 0:
        review = int(review.text.replace('Reviews', '').replace('Review', '').strip())
    else:
        review = 0
    rpwrap = sitter.findAll('span', {'class': 'dv-host-repeat'})
    if len(rpwrap) == 1: # and len(repeat.strip()) > 12:
        repeat = rpwrap[0].text.strip()[:1]
    else:
        repeat = 0
    names = sitter.find('a', {'class': 'dv-host-list-item__body__link'}).text#.strip('\n$').split(' ')
    services = sitter.findAll('div', {'class': 'dv-host-list-item__info dv-host-list-item__services'})[0].text.strip('\n')
    full_star = sitter.findAll('i', {'class':" dv-icon dv-icon__star "})
    ratings = len(full_star)
    half_star = sitter.findAll('i', {'class': " dv-icon dv-icon__star-half "})
    if len(half_star) > 0:
        ratings += .5
    title = soup.find('title').text
    i = title.find(' Dog Boarding')
    title = title[0:i]
    times.append(response_time)
    fees.append(fee)
    reviews.append(review)
    repeats.append(repeat)
    city.append(title)
    name.append(names)
    rating.append(ratings)
    service.append(services)#.text.strip('\n'))
df = pd.DataFrame({'ratings': rating, 'services' : service, 'names': name, 'title':city, 'response_time': times, 'fee':fees, 'review':reviews,'repeat':repeats})

In [23]:
df.head()

,fee,names,ratings,repeat,response_time,review,services,title
0,25,patricia's pet heaven,4.5,9,Few Hours,11,Boarding ...,"Phoenix, Arizona"
1,20,DogsBestFriend,5.0,0,Few Hours,2,"Boarding, ...","Phoenix, Arizona"
2,35,SUZY Q to the Rescue,5.0,2,Day,50,"Boarding, ...","Phoenix, Arizona"
3,25,HappyTails @Camelback East Village,5.0,0,Few Hours,4,"Boarding, ...","Phoenix, Arizona"
4,20,Kelrey's Doggy Dream Stay,4.5,3,Few Hours,3,"Boarding, ...","Phoenix, Arizona"


In [21]:
#eliminate trailing spaces from response_time
df['response_time'] = df['response_time'].apply(lambda x: str(x).strip())

In [34]:
df["fee"] = df["fee"].astype(float)
df["repeat"] = df["repeat"].astype(int)

In [37]:
#df.ix[df['fee']=='', 'fee'] = 0

In [24]:
#Extract columns
fee = df['fee']
repeat = df['repeat']
ratings = df['ratings']

In [40]:
type(df['review'])

pandas.core.series.Series

In [28]:
#The dependent variable
y = np.matrix(fee).transpose()
#The independent variable
x1 = np.matrix(repeat).transpose()
x2 = np.matrix(rating).transpose()

In [32]:
x = np.column_stack([x1, x2])

In [35]:
X = sm.add_constant(x)
model = sm.OLS(y,X)
f = model.fit()
f.summary()

TypeError: coercing to Unicode: need string or buffer, int found

In [22]:
#a = pd.scatter_matrix(df, alpha=.05, figsize=(10,10))
#plt.show()